# Scrapping this
Doing this time split of 14d - 14d doesnt' work that well, in reality we want to be using a few years to predict 2017-type data
Instead we maybe want to be trying to test for periods of 15d in 2017, using all prev data (a few sets of 15d data)

# Why light gbm and goals

Light GBM: because faster, going to spam a lot of features (quick turnaround time)

Goals: add many features, iterate to see what works
    same store/cat:
        1D ago, same store/cat
        7D rolling, same store/cat
        1W ago, same store/cat
        this weekday relative to other weekdays, same store/cat --> over or underperform, quantify
        1M rolling
    
    across store/cat:
        cat performance across all stores, 1d ago (percentile?) --> overperform or underperform

    across larger time:
        last year, how did this same day relate to exactly 1W ago (holiday matching too)

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [17]:
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error

In [4]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
transactions_df = pd.read_csv('data/transactions.csv')
oil_df = pd.read_csv('data/oil.csv')
holidays_df = pd.read_csv('data/holidays_events.csv')
stores_df = pd.read_csv('data/stores.csv')

train_df = train_df.drop(columns='id')
train_df['date'] = pd.to_datetime(train_df['date'])

mask = oil_df.isna().sum(axis=1).astype(bool)
oil_df = oil_df.ffill().bfill()
oil_df['date'] = pd.to_datetime(oil_df['date'])

In [39]:
""" 
Given an initial Df, generates sub train & test dfs
"""
def time_based_split(df, train_duration='30D', test_duration='15D'):
    # Starting point for the first train window
    start_time = df['date'].min()
    end_time = df['date'].max()

    train_duration = pd.Timedelta(train_duration)
    test_duration = pd.Timedelta(test_duration)

    # We keep rolling until we cannot fit another train+test window
    while True:
        train_end_time = start_time + train_duration  # End of the training window
        test_end_time = train_end_time + test_duration  # End of the testing window

        if test_end_time > end_time:
            # If we can't fit a test window in the remaining data, we stop
            break

        # Slice the train/test sets based on the times
        train_df = df[(df['date'] >= start_time) & (df['date'] < train_end_time)]
        test_df = df[(df['date'] >= train_end_time) & (df['date'] < test_end_time)]

        yield (train_df, test_df)

        # Move the start_time to the end of the test window for the next iteration
        start_time = test_end_time

""" 
Runner
Generates dfs using time_based_split
Runs prediction function using predictionFunction
Aggregates results (rmsle), prints/graphs
"""
def runner(df, predictionFunction, train_duration='14D', test_duration='14D'):
    features = df.columns.difference(['sales'])
    target_col = ['sales']

    results = []
    for (train_df, test_df) in time_based_split(df, train_duration, test_duration):
        X_train, y_train = train_df[features], train_df[target_col]
        X_test, y_test = test_df[features], test_df[target_col]

        results.append(predictionFunction(X_train, y_train, X_test, y_test))
        break

    return pd.DataFrame(results)

In [40]:
def fourteenDaysPredictor(X_train, y_train, X_test, y_test):
    train = pd.concat([X_train, y_train], axis=1)
    train['date_14d'] = train['date'] + pd.Timedelta('14D')

    prediction = pd.merge(X_test, train, how='left', left_on='date', right_on='date_14d')['sales']
    rmsle = root_mean_squared_log_error(prediction, y_test)
    print(rmsle)

    return rmsle

In [41]:
res = runner(train_df, fourteenDaysPredictor)
res

0             0.000
1             0.000
2             0.000
3             0.000
4             0.000
             ...   
44457331    374.891
44457332     36.000
44457333      0.000
44457334      0.000
44457335     24.274
Name: sales, Length: 44457336, dtype: float64

ValueError: Found input variables with inconsistent numbers of samples: [44457336, 24948]